In [14]:
import torch

# Function to create a view matrix given rotation (R) and translation (T)
def get_view_matrix(R, T):
    view_matrix = torch.eye(4).cuda()
    view_matrix[:3, :3] = R  # Rotation part
    view_matrix[:3, 3] = T   # Translation part
    return torch.inverse(view_matrix)  # Inverting to get world-to-view

# Function to create a projection matrix given near, far, fovX, and aspect ratio
def get_projection_matrix(znear, zfar, fovX, fovY):
    # Convert degrees to radians manually
    radians_per_degree = torch.tensor(3.14159265358979323846 / 180.0).cuda()  # π/180
    tan_half_fovX = torch.tan(fovX * radians_per_degree / 2)
    tan_half_fovY = torch.tan(fovY * radians_per_degree / 2)

    projection_matrix = torch.zeros((4, 4)).cuda()
    projection_matrix[0, 0] = 1 / tan_half_fovX  # X scaling
    projection_matrix[1, 1] = 1 / tan_half_fovY  # Y scaling
    projection_matrix[2, 2] = -(zfar + znear) / (zfar - znear)  # Z scaling for perspective
    projection_matrix[2, 3] = -(2 * zfar * znear) / (zfar - znear)
    projection_matrix[3, 2] = -1  # Perspective division
    return projection_matrix

# Define parameters
znear = 0.1
zfar = 1000.0
fovX, fovY = 60.0, 60.0
width, height = 800, 600

# Example rotation and translation for view matrix
R = torch.tensor([[1,0,1],[2,3,1],[1,2,0]]).cuda()
T = torch.tensor([1, 0, -5]).cuda()

# Generate view and projection matrices
view_matrix = get_view_matrix(R, T).cuda()
projection_matrix = get_projection_matrix(znear, zfar, fovX, fovY).cuda()

# Combine to get full projection transform
full_proj_transform = projection_matrix @ view_matrix

# Step 2: Define a 3D point in world space and project to 2D
world_point = torch.tensor([2.5, 2.0, 3.0, 1.0]).cuda()  # Homogeneous coordinates
clip_coords = full_proj_transform @ world_point  # Transform to clip space

# Normalize to NDC
ndc_coords = clip_coords[:3] / clip_coords[3]

print(f"Clip coords = {clip_coords}")
print(f"ndc_coords = {ndc_coords}")

# Map NDC to pixel coordinates
pixel_x = (ndc_coords[0].item() * 0.5 + 0.5) * width
pixel_y = (1.0 - (ndc_coords[1].item() * 0.5 + 0.5)) * height

cam_space_val =  view_matrix @ world_point
print(f"cam_space_val = {cam_space_val}")
depth_value = cam_space_val[2] #clip_coords[2].item()  # Using depth from the clip coordinates


print(f"Projected 2D Pixel Coordinates: ({pixel_x}, {pixel_y}), Depth: {depth_value}")

# Step 3: Reverse the process to retrieve 3D coordinates from 2D pixel
# Convert pixel back to NDC
ndc_x = (2 * pixel_x / width) - 1
ndc_y = 1 - (2 * pixel_y / height)

# Use depth value directly
depth_in_camera_space = depth_value  # This should correspond to (view_matrix * world_point)[2]

# Create clip space coordinates with the known depth
clip_coords_reconstructed = torch.tensor([ndc_x, ndc_y, depth_in_camera_space, 1.0]).cuda()

# Back-project to camera space
full_proj_inv = torch.inverse(full_proj_transform)

# Check intermediate values for debugging
print(f"Clip Coordinates for Reconstruction: {clip_coords_reconstructed}")

world_point_reconstructed = full_proj_inv @ clip_coords_reconstructed

# Normalize homogeneous coordinates without in-place division
normalization_factor = world_point_reconstructed[3]
world_point_reconstructed = world_point_reconstructed / normalization_factor  # Divide by the normalization factor

print(f"Reconstructed 3D World Coordinates: {world_point_reconstructed[:3]}")
print(f"Original 3D World Coordinates: {world_point[:3]}")

# Check if original and reconstructed points match
print(f"Comparison: {world_point[:3]} vs {world_point_reconstructed[:3]}")
assert torch.allclose(world_point[:3], world_point_reconstructed[:3], atol=1e-5), "Reconstruction failed"
print("3D Reconstruction from 2D coordinates is successful!")


Clip coords = tensor([ 39.8372, -12.9904,  21.3043,  21.5000], device='cuda:0')
ndc_coords = tensor([ 1.8529, -0.6042,  0.9909], device='cuda:0')
cam_space_val = tensor([ 23.0000,  -7.5000, -21.5000,   1.0000], device='cuda:0')
Projected 2D Pixel Coordinates: (1141.1565780639648, 481.26112818717957), Depth: -21.500001907348633
Clip Coordinates for Reconstruction: tensor([  1.8529,  -0.6042, -21.5000,   1.0000], device='cuda:0')
Reconstructed 3D World Coordinates: tensor([ 1.0006e+00,  8.2546e-04, -4.9967e+00], device='cuda:0')
Original 3D World Coordinates: tensor([2.5000, 2.0000, 3.0000], device='cuda:0')
Comparison: tensor([2.5000, 2.0000, 3.0000], device='cuda:0') vs tensor([ 1.0006e+00,  8.2546e-04, -4.9967e+00], device='cuda:0')


AssertionError: Reconstruction failed

In [8]:
import torch
import numpy as np

# Function to create a view matrix given rotation (R) and translation (T)
def get_view_matrix(R, T):
    view_matrix = torch.eye(4)
    view_matrix[:3, :3] = R  # Rotation part
    view_matrix[:3, 3] = T   # Translation part
    return torch.inverse(view_matrix)  # Inverting to get world-to-view

# Function to create a projection matrix given near, far, fovX, and aspect ratio
def get_projection_matrix(znear, zfar, fovX, fovY):
    # Convert degrees to radians manually
    radians_per_degree = torch.tensor(3.14 / 180.0) # π/180
    tan_half_fovX = torch.tan(fovX * radians_per_degree / 2)
    tan_half_fovY = torch.tan(fovY * radians_per_degree / 2)

    projection_matrix = torch.zeros((4, 4))
    projection_matrix[0, 0] = 1 / tan_half_fovX  # X scaling
    projection_matrix[1, 1] = 1 / tan_half_fovY  # Y scaling
    projection_matrix[2, 2] = -(zfar + znear) / (zfar - znear)  # Z scaling for perspective
    projection_matrix[2, 3] = -(2 * zfar * znear) / (zfar - znear)
    projection_matrix[3, 2] = -1  # Perspective division
    return projection_matrix


znear = 0.1
zfar = 1000.0
fovX, fovY = 80.0, 90.0
width, height = 800, 600


R = torch.tensor([[1,0,2],[0,1,2],[1,0,1]])
T = torch.tensor([2, 1, -5])


view_matrix = get_view_matrix(R, T)
projection_matrix = get_projection_matrix(znear, zfar, fovX, fovY)


full_proj_transform = projection_matrix @ view_matrix


world_point = torch.tensor([15.0, 20.0, 4.0, 1.0]) 
clip_coords = full_proj_transform @ world_point 
print(f"clip_coords ={clip_coords}")

# Normalize to NDC
ndc_coords = clip_coords[:3] / clip_coords[3]

# Map NDC to pixel coordinates
pixel_x = (ndc_coords[0].item() * 0.5 + 0.5) * width
pixel_y = (1.0 - (ndc_coords[1].item() * 0.5 + 0.5)) * height

cam_space_val = view_matrix @ world_point
print(f"cam_space_val = {cam_space_val}")
depth_value = cam_space_val[2].item() 

print(f"Projected 2D Pixel Coordinates: ({pixel_x}, {pixel_y}), Depth: {depth_value}")

ndc_x = (2 * pixel_x / width) - 1
ndc_y = 1 - (2 * pixel_y / height)


depth_in_camera_space = depth_value  # Using positive depth for reconstruction


clip_coords_reconstructed = -1*torch.tensor([ndc_x, ndc_y, 1.0, 1.0])*depth_in_camera_space

# Back-project to camera space
full_proj_inv = torch.inverse(full_proj_transform)


print(f"Clip Coordinates for Reconstruction: {clip_coords_reconstructed}")


print(f"view_matrix = {view_matrix}")
print(f"projection_matrix = {projection_matrix}")
print(f"full_proj_transform = {full_proj_transform}")
print(f"full_proj_inv = {full_proj_inv}")

p = full_proj_transform[torch.tensor([0,1,3])][:,:3]
print(p)

b = clip_coords_reconstructed[:3]-full_proj_transform[torch.tensor([0,1,3])][:,3]
print(b)
x, res, r, s = np.linalg.lstsq(p, b)

world_point_reconstructed_rounded = torch.round(torch.tensor(x)*1000)/1000

print(f"Reconstructed 3D World Coordinates: {world_point_reconstructed_rounded}")
print(f"Original 3D World Coordinates: {world_point[:3]}")

# Check if original and reconstructed points match with a tolerance
tolerance = 1e-2  # Allow a small error margin
comparison = torch.abs(world_point[:3] - world_point_reconstructed_rounded)
print(f"Comparison: {comparison}")

# Check if the difference is within the allowed tolerance
assert torch.all(comparison < tolerance), "Reconstruction failed"
print("3D Reconstruction from 2D coordinates is successful!")


clip_coords =tensor([ 5.9631, 11.0088, -4.2008, -4.0000])
cam_space_val = tensor([ 5., 11.,  4.,  1.])
Projected 2D Pixel Coordinates: (-196.3052749633789, 1125.657033920288), Depth: 4.0
Clip Coordinates for Reconstruction: tensor([ 5.9631, 11.0088, -4.0000, -4.0000])
view_matrix = tensor([[-1., -0.,  2., 12.],
        [-2.,  1.,  2., 13.],
        [ 1.,  0., -1., -7.],
        [ 0.,  0.,  0.,  1.]])
projection_matrix = tensor([[ 1.1926,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  1.0008,  0.0000,  0.0000],
        [ 0.0000,  0.0000, -1.0002, -0.2000],
        [ 0.0000,  0.0000, -1.0000,  0.0000]])
full_proj_transform = tensor([[-1.1926,  0.0000,  2.3852, 14.3113],
        [-2.0016,  1.0008,  2.0016, 13.0104],
        [-1.0002,  0.0000,  1.0002,  6.8014],
        [-1.0000,  0.0000,  1.0000,  7.0000]])
full_proj_inv = tensor([[ 8.3850e-01,  2.3842e-07, -9.9990e+00,  8.0010e+00],
        [ 2.3842e-07,  9.9920e-01, -4.9995e+00,  3.0005e+00],
        [ 8.3849e-01,  9.5367e-07,  2.4997e+

/tmp/ipykernel_2268036/2045222530.py:87: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  x, res, r, s = np.linalg.lstsq(p, b)


In [9]:
import torch 
import numpy as np

def pix2ndc(p,s):
    return (2*p +1)/s -1

h, w = 1921, 1073

projection_full = torch.tensor([[ 1.2347, -0.3155,  0.6432,  0.6431],
        [ 1.0517,  0.7530, -0.2665, -0.2664],
        [-0.7158,  0.5621,  0.7180,  0.7179],
        [ 0.8998, -1.8877,  3.9454,  3.9551]])
means_2d = torch.tensor([[   0.0000,    0.0000],
        [   0.0000,    0.0000],
        [   0.0000,    0.0000],
        [ 566.0210, 1040.1997],
        [   0.0000,    0.0000],
        [   0.0000,    0.0000],
        [ 480.4791, 1019.8796],
        [   0.0000,    0.0000],
        [ 180.9700, 1055.9210],
        [   0.0000,    0.0000]], requires_grad=True)

means_2d_new = torch.ones_like(means_2d)
means_2d_new[:,0], means_2d_new[:,1] = pix2ndc(means_2d[:,0],w), pix2ndc(means_2d[:,1],h)

print(f"new means_2d = \n {means_2d_new}")

depths = torch.tensor([0.0000, 0.0000, 0.0000, 5.4124, 0.0000, 0.0000, 3.6705, 0.0000, 9.4935,
        0.0000])
means_3d = torch.tensor([[-1.9686, -5.0952,  8.4925],
        [-7.0956, -0.8330,  3.4324],
        [-3.6041,  3.3491,  2.8043],
        [-0.0486,  1.2050,  2.5206],
        [-7.4773, -0.5681,  3.6143],
        [-7.0176, -0.3949,  3.8820],
        [-1.3654,  1.2697,  1.2979],
        [-6.6321, -2.4209,  0.9914],
        [-0.1703, -1.7058,  7.2341],
        [-1.7995, -5.9542,  7.3705]])

def get_mean_3d(mean_2d, depth, projection_full):
    size = means_2d.shape[0]
    p = projection_full.T[torch.tensor([0,1,3])][:,:3]
    print(f"p \n {p}")
    b = torch.ones(size,3)
    b[:,:2], b[:,2] = mean_2d.detach()*depth.reshape(size,1), depth
    print(f"b \n {b}")
    b = b - projection_full.T[torch.tensor([0,1,3])][:,3]
    print(f"b \n {b}")
    X = np.ones((size,3))
    for i in range(size):
        X[i], res, r, s = np.linalg.lstsq(p, b[i])
    print(f"Reconstructed 3d \n {X}")
    return torch.tensor(X, requires_grad=True) 

x = get_mean_3d(means_2d_new, depths, projection_full)

new means_2d = 
 tensor([[-0.9991, -0.9995],
        [-0.9991, -0.9995],
        [-0.9991, -0.9995],
        [ 0.0560,  0.0835],
        [-0.9991, -0.9995],
        [-0.9991, -0.9995],
        [-0.1035,  0.0623],
        [-0.9991, -0.9995],
        [-0.6618,  0.0999],
        [-0.9991, -0.9995]], grad_fn=<CopySlices>)
p 
 tensor([[ 1.2347,  1.0517, -0.7158],
        [-0.3155,  0.7530,  0.5621],
        [ 0.6431, -0.2664,  0.7179]])
b 
 tensor([[-0.0000, -0.0000,  0.0000],
        [-0.0000, -0.0000,  0.0000],
        [-0.0000, -0.0000,  0.0000],
        [ 0.3029,  0.4519,  5.4124],
        [-0.0000, -0.0000,  0.0000],
        [-0.0000, -0.0000,  0.0000],
        [-0.3798,  0.2288,  3.6705],
        [-0.0000, -0.0000,  0.0000],
        [-6.2823,  0.9481,  9.4935],
        [-0.0000, -0.0000,  0.0000]])
b 
 tensor([[-0.8998,  1.8877, -3.9551],
        [-0.8998,  1.8877, -3.9551],
        [-0.8998,  1.8877, -3.9551],
        [-0.5969,  2.3396,  1.4573],
        [-0.8998,  1.8877, -3.9551],


/tmp/ipykernel_2268036/2756429043.py:53: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  X[i], res, r, s = np.linalg.lstsq(p, b[i])


In [10]:

def get_mean_3d(mean_2d, depth, projection_full, w, h):
    """
    Inputs:-
        mean_2d - 2d prixel centroid values of blobs
        depth - Average depth tensor (use knn to get the neighbours and get an average depth value for blobs)
        projection_full - Projection_full matrix of camera viewpoint (projection_mat @ view_mat)
        w - image width
        h - image hight
    Output
        X - 3d coordinates w.r.t world coordinates
    """
    size = mean_2d.shape[0]
    # Converting pixel values to ndc
    means_2d_new = torch.ones_like(mean_2d)
    means_2d_new[:,0], means_2d_new[:,1] = pix2ndc(mean_2d[:,0],w), pix2ndc(mean_2d[:,1],h)

    p = projection_full.T[torch.tensor([0,1,3])][:,:3]  # In 3d gaussian splatting, they have used column prioritizing
    b = torch.ones(size,3)
    b[:,:2] = means_2d_new.detach()*depth.reshape(size,1)
    b[:,2] =  depth
    b = b - projection_full.T[torch.tensor([0,1,3])][:,3]
    X = np.ones((size,3))
    for i in range(size):
        X[i], res, r, s = torch.linalg.lstsq(p, b[i], rcond=None)
    return torch.tensor(X, requires_grad=True) 

x = get_mean_3d(means_2d, depths, projection_full,w,h)

print(x)

tensor([[-3.5035,  2.1995, -1.5546],
        [-3.5035,  2.1995, -1.5546],
        [-3.5035,  2.1995, -1.5546],
        [-0.0486,  1.2051,  2.5207],
        [-3.5035,  2.1995, -1.5546],
        [-3.5035,  2.1995, -1.5546],
        [-1.3655,  1.2698,  1.2980],
        [-3.5035,  2.1995, -1.5546],
        [-0.1702, -1.7056,  7.2343],
        [-3.5035,  2.1995, -1.5546]], dtype=torch.float64, requires_grad=True)


In [11]:
means_3d = torch.tensor([[-1.9686, -5.0952,  8.4925],
        [-7.0956, -0.8330,  3.4324],
        [-3.6041,  3.3491,  2.8043],
        [-0.0486,  1.2050,  2.5206],
        [-7.4773, -0.5681,  3.6143],
        [-7.0176, -0.3949,  3.8820],
        [-1.3654,  1.2697,  1.2979],
        [-6.6321, -2.4209,  0.9914],
        [-0.1703, -1.7058,  7.2341],
        [-1.7995, -5.9542,  7.3705]])

In [12]:
import time


def get_mean_3d_cuda(blobs_xy, depth, projection_full, w, h):
    """
    Inputs:
        blobs_xy - 2D pixel centroid values of blobs
        depth - Average depth tensor (use knn to get neighbors and get an average depth value for blobs)
        projection_full - Projection matrix of camera viewpoint (projection_mat @ view_mat)
        w - Image width
        h - Image height
    Output:
        X - 3D coordinates with respect to world coordinates
    """
    device = blobs_xy.device  # Assuming blobs_xy, depth, and projection_full are on the same device (CUDA)

    # Number of blobs
    size = blobs_xy.shape[0]

    # Converting pixel values to NDC (Normalized Device Coordinates)
    blobs_xy_new = torch.ones_like(blobs_xy, device=device)
    blobs_xy_new[:, 0], blobs_xy_new[:, 1] = pix2ndc(blobs_xy[:, 0], w), pix2ndc(blobs_xy[:, 1], h)

    # Preparing matrices for Ax = b
    p = projection_full.T[torch.tensor([0, 1, 3], device=device)][:, :3]  # Column prioritizing
    b = torch.ones(size, 3, device=device)  # b matrix in Ax = b
    b[:, :2] = blobs_xy_new * depth.view(size, 1)
    b[:, 2] = depth

    # Adjust b by subtracting translation part of projection
    b = b - projection_full.T[torch.tensor([0, 1, 3], device=device)][:, 3]

    # Solve Ax = b in batch mode
    # p needs to be expanded to match the size of b for batch lstsq
    p_batch = p.unsqueeze(0).expand(size, -1, -1)  # Expanding p to shape (size, 3, 3)
    b_batch = b.unsqueeze(-1)  # Making b shape (size, 3, 1)

    # Use torch.linalg.lstsq on CUDA with the `gels` driver (default for CUDA)
    X= torch.linalg.lstsq(p_batch, b_batch).solution

    # Remove the extra dimension added for lstsq compatibility
    X = X.squeeze(-1)

    return X


start_time_1 = time.time()
x = get_mean_3d(means_2d, depths, projection_full,w,h)
end_time_1 = time.time()

execution_time = end_time_1 - start_time_1
print("Execution time 1:", execution_time, "seconds")
print(x)

means_2d = means_2d.to(device='cuda')
depths = depths.to(device='cuda')
projection_full = projection_full.to(device = 'cuda')

start_time_2 = time.time()
x = get_mean_3d_cuda(means_2d, depths, projection_full,w,h)
end_time_2 = time.time()

execution_time = end_time_2 - start_time_2
print("Execution time 2:", execution_time, "seconds")

print(x)

Execution time 1: 0.001003265380859375 seconds
tensor([[-3.5035,  2.1995, -1.5546],
        [-3.5035,  2.1995, -1.5546],
        [-3.5035,  2.1995, -1.5546],
        [-0.0486,  1.2051,  2.5207],
        [-3.5035,  2.1995, -1.5546],
        [-3.5035,  2.1995, -1.5546],
        [-1.3655,  1.2698,  1.2980],
        [-3.5035,  2.1995, -1.5546],
        [-0.1702, -1.7056,  7.2343],
        [-3.5035,  2.1995, -1.5546]], dtype=torch.float64, requires_grad=True)
Execution time 2: 0.0007510185241699219 seconds
tensor([[-3.5035,  2.1995, -1.5546],
        [-3.5035,  2.1995, -1.5546],
        [-3.5035,  2.1995, -1.5546],
        [-0.0486,  1.2051,  2.5207],
        [-3.5035,  2.1995, -1.5546],
        [-3.5035,  2.1995, -1.5546],
        [-1.3655,  1.2698,  1.2980],
        [-3.5035,  2.1995, -1.5546],
        [-0.1702, -1.7056,  7.2343],
        [-3.5035,  2.1995, -1.5546]], device='cuda:0',
       grad_fn=<SqueezeBackward1>)
